In [2]:
import os

In [2]:
%pwd

'c:\\Users\\hardi\\OneDrive - wipro\\Desktop\\DL ops HS\\vinipro\\research'

In [17]:
os.chdir('../')

In [5]:
os.getcwd

<function nt.getcwd()>

In [6]:
%pwd

'c:\\Users\\hardi\\OneDrive - wipro\\Desktop\\DL ops HS\\vinipro\\research'

In [7]:
os.chdir("../")

In [8]:
%pwd

'c:\\Users\\hardi\\OneDrive - wipro\\Desktop\\DL ops HS\\vinipro'

In [9]:
# dataclass ---> it import a pyton tool called dataclass that helps you to create classes that only store data. 
# it reduces the amount of code you need to write (no need to manually write __init__)

# pathlib -->  it imports the path class , which helps you work with file and folder path easily and in a clean way for all os.

# @dataclass(frozen=true) ---> it is a type of decorator that tells python to dataclass means we cannot change the 
# vaule once the object is created. it makes the configuration safe from accidental changes.

In [10]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path 
    source_URL: str 
    local_data_file: Path 
    unzip_dir: Path

In [11]:

from src.cnnclassifier.constants import * 
from src.cnnclassifier.utils.common import read_yaml , create_directories 

In [12]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath) 

        create_directories([self.config.artifacts_root]) 

    def get_data_ingestion_config(self):
        config = self.config.data_ingestion 

        create_directories([config.root_dir]) 
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir = config.unzip_dir 

        )

        return data_ingestion_config

In [14]:

import os 
import urllib.request as request 
import zipfile 
from src.cnnclassifier import logger 
from src.cnnclassifier.utils.common import get_size 

In [15]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config 

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )

            logger.info(f"{filename} download! with following info: \n {headers}") 
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path , exist_ok=True) 
        with zipfile.ZipFile(self.config.local_data_file , 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [18]:

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config() 
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file() 
except Exception as e :
    raise e

[2025-08-19 01:14:37,046 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2025-08-19 01:14:37,069 : INFO : common : yaml file: params.yaml loaded successfully]
[2025-08-19 01:14:37,071 : INFO : common : created directory at: artifacts]
[2025-08-19 01:14:37,073 : INFO : common : created directory at: artifacts/data_ingestion]
[2025-08-19 01:14:46,235 : INFO : 3399376260 : artifacts/data_ingestion/data.zip download! with following info: 
 Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4645:3D855E:69FEA:EA61B:68A382A5
Accept-Ranges: bytes
Date: Mon, 18 Aug 2025 19:44:39 GMT
Via: 1.1 varnish
X-Served-By: cache-bom-vanm